# Basic Arnoldi

From ChatGPT precusrsor

In [38]:
using LinearAlgebra

function Arnoldi1(A, v, m)
    n = length(v)
    Q = Matrix{Float64}(undef, n, m+1)  # Matrix to store orthogonalized vectors
    H = zeros(Float64, m+1, m)          # Upper Hessenberg matrix

    # Skipping normalizing initial vector and store it in Q
    Q[:, 1] = v

    for k = 1:m
        # Arnoldi iteration
        w = A*Q[:, k]
        for j = 1:k
            # Orthogonalization using modified Gram-Schmidt
            H[j, k] = dot(Q[:, j], w)
            w -= H[j, k] * Q[:, j]
        end
        H[k+1, k] = norm(w)
        Q[:, k+1] = w / H[k+1, k]
    end
    return Q, H
end

function Arnoldi2(A, v, m)
    n = length(v)
    Q = Matrix{Float64}(undef, n, m+1)  # Matrix to store orthogonalized vectors
    H = zeros(Float64, m+1, m)          # Upper Hessenberg matrix

    # Skipping normalizing initial vector and store it in Q
    Q[:, 1] = v

    for k = 1:m
        # Arnoldi iteration
        w = A*Q[:, k]
        for j = 1:k
            # Orthogonalization using modified Gram-Schmidt
            H[j, k] = dot(Q[:, j], w)
            w -= H[j, k] * Q[:, j]
        end
        H[k+1, k] = norm(w)
        Q[:, k+1] = w / H[k+1, k]
    end
    return H[m+1,m]
end

function Arnoldi3(v)
    n = length(v)
    Q = Matrix{Float64}(undef, n, m+1)  # Matrix to store orthogonalized vectors
    H = zeros(Float64, m+1, m)          # Upper Hessenberg matrix

    # Skipping normalizing initial vector and store it in Q
    Q[:, 1] = v

    for k = 1:m
        # Arnoldi iteration
        w = A*Q[:, k]
        for j = 1:k
            # Orthogonalization using modified Gram-Schmidt
            H[j, k] = dot(Q[:, j], w)
            w -= H[j, k] * Q[:, j]
        end
        H[k+1, k] = norm(w)
        Q[:, k+1] = w / H[k+1, k]
    end
    return H[m+1,m]
end

Arnoldi3 (generic function with 1 method)

Testing standard input output form using a package from https://github.com/CHLOzzz/MMGet/blob/main/README.md to grab a MatrixMarket matrix.  keep_files = true preserves files. 

In [48]:
using MMGet
A = mmget("https://math.nist.gov/pub/MatrixMarket2/Harwell-Boeing/bcsstruc1/bcsstk01.mtx.gz", keep_files = false)
m = 4
v=rand(size(A,1)); v=v/norm(v)
@time (Q, H) = Arnoldi1(A,v,m)
(norm(A*Q[:,1:m] - Q*H)/norm(A),norm(Q'*Q-I),H[m+1,m])

  0.000065 seconds (57 allocations: 25.047 KiB)


(5.598770116509044e-17, 4.195961757412974e-15, 5.041896356673027e8)

Testing reduced output form.  Only bottom entry of H. 

In [44]:
using MMGet
A = mmget("https://math.nist.gov/pub/MatrixMarket2/Harwell-Boeing/bcsstruc1/bcsstk01.mtx.gz", keep_files = false)
m = 4
v=rand(size(A,1)); v=v/norm(v)
@time Arnoldi2(A,v,m)

  0.000051 seconds (55 allocations: 24.969 KiB)


5.373380082111145e8

Testing reduced inout form. 

In [50]:
using MMGet
A = mmget("https://math.nist.gov/pub/MatrixMarket2/Harwell-Boeing/bcsstruc1/bcsstk01.mtx.gz", keep_files = false)
m = 4
v=rand(size(A,1)); v=v/norm(v)
@time Arnoldi3(v)

  0.000151 seconds (326 allocations: 30.000 KiB)


5.480437507309493e8

testing ForwardDiff 

In [75]:
using ForwardDiff
function f(x)
    0.5*x'*x
end
b=rand(5)
@time f(b)
@time y=ForwardDiff.gradient(f,b)
display(norm(y-b))
@time y=ForwardDiff.hessian(f,b)

  0.009211 seconds (2.38 k allocations: 179.688 KiB, 98.23% compilation time)
  0.013132 seconds (6.17 k allocations: 429.750 KiB, 99.75% compilation time: 100% of which was recompilation)


0.0

  0.922469 seconds (1.88 M allocations: 115.893 MiB, 2.71% gc time, 99.97% compilation time)


5×5 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0

In [101]:
using MMGet, LinearAlgebra, SparseArrays
A = mmget("https://math.nist.gov/pub/MatrixMarket2/Harwell-Boeing/bcsstruc1/bcsstk01.mtx.gz", keep_files = false)
n=size(A,1)
(m,s) = (4,3)
bG = (qr(rand(n,m+1)).Q)*Matrix(1.0I, m+1, m+1)
b=bG[:,1]
G=bG[:,2:s+1]
function f(y)
    Arnoldi2(A, b + G*y,m)
end
(f(zeros(s)),f(0.1*rand(s)))

(4.9978955538653266e8, 5.178845003143101e8)

In [131]:
using MMGet, LinearAlgebra, SparseArrays
A = mmget("https://math.nist.gov/pub/MatrixMarket2/Harwell-Boeing/bcsstruc1/bcsstk01.mtx.gz", keep_files = false)
n=size(A,1)
#A=rand(n,n)
function f(y)
    
    n = 48
    Q = Matrix{Float64}(undef, n, m+1)  # Matrix to store orthogonalized vectors
    H = zeros(Float64, m+1, m)          # Upper Hessenberg matrix

    # Normalizing initial vector and store it in Q
    v = b + G*y
    Q[:, 1] = v/sqrt(v'*v)
    for k = 1:m
        # Arnoldi iteration
        w = A*Q[:, k]
        for j = 1:k
            # Orthogonalization using modified Gram-Schmidt
            H[j, k] = Q[:, j]'*w
            w -= H[j, k] * Q[:, j]
        end
        H[k+1, k] = sqrt(w'*w)
        Q[:, k+1] = w / H[k+1, k]
    end
    return H[m+1,m]
end

f (generic function with 1 method)

In [133]:
y=zeros(s)
f(y)
ForwardDiff.gradient(f,y)

LoadError: MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{typeof(f), Float64}, Float64, 3})

[0mClosest candidates are:
[0m  (::Type{T})(::Real, [91m::RoundingMode[39m) where T<:AbstractFloat
[0m[90m   @[39m [90mBase[39m [90m[4mrounding.jl:207[24m[39m
[0m  (::Type{T})(::T) where T<:Number
[0m[90m   @[39m [90mCore[39m [90m[4mboot.jl:792[24m[39m
[0m  Float64([91m::UInt8[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mfloat.jl:165[24m[39m
[0m  ...
